<a href="https://colab.research.google.com/github/jonG312/cuda_ejercicios/blob/main/parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
!apt install nvidia-cuda-toolkit
%load_ext nvcc4jupyter

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java libatk-wrapper-java-jni
  libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10 libcufftw10
  libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1 libgail-common libgail18
  libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist1

In [2]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <time.h>

using namespace std;

const int N = 1000;

#define Bsize_addition 256
#define Bsize_minimum 128

//**************************************************************************
// Vector addition kernel
//**************************************************************************
__global__ void add_arrays_gpu(float *in1, float *in2, float *out, int N)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N)
        out[idx] = in1[idx] + in2[idx];
}
//**************************************************************************

//**************************************************************************
// Vector sum kernel
//**************************************************************************
__global__ void reduceSum(float *V_in, float *V_out, const int N)
{
    extern __shared__ float sdata[];

    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    sdata[tid] = ((i < N) ? V_in[i] : 0.0f);
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s >>= 1)
    {
        if (tid < s)
            sdata[tid] += sdata[tid + s];
        __syncthreads();
    }
    if (tid == 0)
        V_out[blockIdx.x] = sdata[0];
}

//**************************************************************************
int main()
//**************************************************************************

{
    srand(time(NULL));
    /* pointers to host memory */
    float *a, *b, *c;
    /* pointers to device memory */
    float *a_d, *b_d, *c_d;
    int i;

    /* Allocate arrays a, b and c on host*/
    a = (float *)malloc(N * sizeof(float));
    b = (float *)malloc(N * sizeof(float));
    c = (float *)malloc(N * sizeof(float));

    /* Allocate arrays a_d, b_d and c_d on device*/
    cudaMalloc((void **)&a_d, sizeof(float) * N);
    cudaMalloc((void **)&b_d, sizeof(float) * N);
    cudaMalloc((void **)&c_d, sizeof(float) * N);

    /* Initialize arrays a and b */
    for (i = 0; i < N; i++)
    {
        a[i] = (float)(rand() % 1000);
        b[i] = (float)(rand() % 45);
    }

    /* Copy data from host memory to device memory */
    cudaMemcpy(a_d, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(b_d, b, sizeof(float) * N, cudaMemcpyHostToDevice);

    /* Compute the execution configuration */
    dim3 dimBlock(Bsize_addition);
    dim3 dimGrid(ceil((float)(N) / (float)dimBlock.x));

    /* Add arrays a and b, store result in c */
    add_arrays_gpu<<<dimGrid, dimBlock>>>(a_d, b_d, c_d, N);

    //**************************************************
    // Block reduction on GPU to obtain partial sum
    //**************************************************
    dim3 threadsPerBlock(Bsize_minimum, 1);
    dim3 numBlocks(ceil((float)(N) / threadsPerBlock.x), 1);

    // Sum vector on CPU
    float *vsum;
    vsum = (float *)malloc(numBlocks.x * sizeof(float));

    // Sum vector to be computed on GPU
    float *vsum_d;
    cudaMalloc((void **)&vsum_d, sizeof(float) * numBlocks.x);

    int smemSize = threadsPerBlock.x * sizeof(float);
    // Kernel launch to compute Sum Vector
    reduceSum<<<numBlocks, threadsPerBlock, smemSize>>>(c_d, vsum_d, N);

    /* Copy data from device memory to host memory */
    cudaMemcpy(vsum, vsum_d, numBlocks.x * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(c, c_d, sizeof(float) * N, cudaMemcpyDeviceToHost);

    // Perform final reduction in CPU
    float sum_gpu = 0.0f;
    cout << "Performing final reduction of the partial results on CPU:" << endl;
    for (int i = 0; i < numBlocks.x; i++)
    {
        sum_gpu += vsum[i];
    }

    cout << endl
         << "... Sum on GPU =" << sum_gpu << "               ";

    //***********************
    // Compute sum on CPU
    //***********************
    float sum_cpu = 0.0f;
    for (i = 0; i < N; i++)
    {
        sum_cpu += c[i];
    }

    cout << ".... Sum on CPU=" << sum_cpu << endl;

    // Calculate mean
    float mean_gpu = sum_gpu / N;
    float mean_cpu = sum_cpu / N;

    cout << "Mean on GPU: " << mean_gpu << endl;
    cout << "Mean on CPU: " << mean_cpu << endl;

    /* Free the memory */
    free(a);
    free(b);
    free(c);
    free(vsum);
    cudaFree(a_d);
    cudaFree(b_d);
    cudaFree(c_d);
    cudaFree(vsum_d);
}

Performing final reduction of the partial results on CPU:

... Sum on GPU =529790               .... Sum on CPU=529790
Mean on GPU: 529.79
Mean on CPU: 529.79



## Implementación con Memoria Compartida para el Cálculo del Vector C

In [6]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <math.h>

#define BLOCK_SIZE 256

using namespace std;

__global__ void vectorOperationWithSharedMemory(float *A, float *B, float *C, int N, int blockSize) {
    __shared__ float partialSum[BLOCK_SIZE];

    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int localIdx = threadIdx.x;

    float sum = 0.0f;
    for (int i = tid; i < N; i += blockSize) {
        // Modificamos la expresión para evitar problemas de tipos de datos
        if ((int)(ceil(A[i])) % 2 == 0) {
            sum += A[i] + B[i];
        } else {
            sum += A[i] - B[i];
        }
    }

    partialSum[localIdx] = sum;

    __syncthreads();

    // Reducción dentro del bloque
    for (int stride = blockSize / 2; stride > 0; stride >>= 1) {
        if (localIdx < stride) {
            partialSum[localIdx] += partialSum[localIdx + stride];
        }
        __syncthreads();
    }

    // Escribir el resultado en la memoria global
    if (localIdx == 0) {
        C[blockIdx.x] = partialSum[0];
    }
}

int main() {
    srand(time(NULL));

    const int N = 1000;
    const int numBlocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;

    h_A = (float*)malloc(N * sizeof(float));
    h_B = (float*)malloc(N * sizeof(float));
    h_C = (float*)malloc(numBlocks * sizeof(float));

    cudaMalloc((void**)&d_A, N * sizeof(float));
    cudaMalloc((void**)&d_B, N * sizeof(float));
    cudaMalloc((void**)&d_C, numBlocks * sizeof(float));

    for (int i = 0; i < N; i++) {
        h_A[i] = (float)(1.5 * (1 + (5 * i) % 7) / (1 + i % 5));
        h_B[i] = (float)(2.0 * (2 + i % 5) / (1 + i % 7));
    }

    cudaMemcpy(d_A, h_A, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, N * sizeof(float), cudaMemcpyHostToDevice);

    vectorOperationWithSharedMemory<<<numBlocks, BLOCK_SIZE>>>(d_A, d_B, d_C, N, BLOCK_SIZE);

    cudaMemcpy(h_C, d_C, numBlocks * sizeof(float), cudaMemcpyDeviceToHost);

    float sum = 0.0f;
    for (int i = 0; i < numBlocks; i++) {
        sum += h_C[i];
    }

    cout << "Sum Vector C: " << sum << endl;

    free(h_A);
    free(h_B);
    free(h_C);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Sum: 4154.16



## Implementación sin Memoria Compartida para el Cálculo del Vector C

In [8]:
%%cuda

#include <stdio.h>
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

// Kernel para el cálculo del vector C sin utilizar memoria compartida
__global__ void vectorOperationWithoutSharedMemory(float *A, float *B, float *C, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Modificamos la expresión para evitar problemas de tipos de datos
    if ((int)(ceil(A[idx])) % 2 == 0) {
        C[idx] = A[idx] + B[idx];
    } else {
        C[idx] = A[idx] - B[idx];
    }
}

int main() {
    const int N = 1000;
    const int BLOCK_SIZE = 256;

    float *h_A, *h_B, *h_C;
    float *d_A, *d_B, *d_C;

    h_A = (float*)malloc(N * sizeof(float));
    h_B = (float*)malloc(N * sizeof(float));
    h_C = (float*)malloc(N * sizeof(float));

    cudaMalloc((void**)&d_A, N * sizeof(float));
    cudaMalloc((void**)&d_B, N * sizeof(float));
    cudaMalloc((void**)&d_C, N * sizeof(float));

    for (int i = 0; i < N; i++) {
        h_A[i] = (float)(1.5 * (1 + (5 * i) % 7) / (1 + i % 5));
        h_B[i] = (float)(2.0 * (2 + i % 5) / (1 + i % 7));
    }

    cudaMemcpy(d_A, h_A, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, N * sizeof(float), cudaMemcpyHostToDevice);

    dim3 blockSize(BLOCK_SIZE);
    dim3 gridSize((N + BLOCK_SIZE - 1) / BLOCK_SIZE);

    vectorOperationWithoutSharedMemory<<<gridSize, blockSize>>>(d_A, d_B, d_C, N);

    cudaMemcpy(h_C, d_C, N * sizeof(float), cudaMemcpyDeviceToHost);

    // Mostrar el vector C calculado
    cout << "Vector C:" << endl;
    for (int i = 0; i < N; i++) {
        cout << h_C[i] << " ";
    }
    cout << endl;

    free(h_A);
    free(h_B);
    free(h_C);

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Vector C:
5.5 1.5 4.66667 -1.75 -0.3 8.16667 1.39286 -7.5 -2.75 5.2 2 6.45 1.16667 2.55357 -11.7 7 1 -1 0.625 3.5 3.92857 -5.25 -1 4.83333 -2.4 9.7 4.75 2.64286 -9.625 7.8 7.33333 3 5.1 3.54167 -0.814286 5.5 1.5 4.66667 -1.75 -0.3 8.16667 1.39286 -7.5 -2.75 5.2 2 6.45 1.16667 2.55357 -11.7 7 1 -1 0.625 3.5 3.92857 -5.25 -1 4.83333 -2.4 9.7 4.75 2.64286 -9.625 7.8 7.33333 3 5.1 3.54167 -0.814286 5.5 1.5 4.66667 -1.75 -0.3 8.16667 1.39286 -7.5 -2.75 5.2 2 6.45 1.16667 2.55357 -11.7 7 1 -1 0.625 3.5 3.92857 -5.25 -1 4.83333 -2.4 9.7 4.75 2.64286 -9.625 7.8 7.33333 3 5.1 3.54167 -0.814286 5.5 1.5 4.66667 -1.75 -0.3 8.16667 1.39286 -7.5 -2.75 5.2 2 6.45 1.16667 2.55357 -11.7 7 1 -1 0.625 3.5 3.92857 -5.25 -1 4.83333 -2.4 9.7 4.75 2.64286 -9.625 7.8 7.33333 3 5.1 3.54167 -0.814286 5.5 1.5 4.66667 -1.75 -0.3 8.16667 1.39286 -7.5 -2.75 5.2 2 6.45 1.16667 2.55357 -11.7 7 1 -1 0.625 3.5 3.92857 -5.25 -1 4.83333 -2.4 9.7 4.75 2.64286 -9.625 7.8 7.33333 3 5.1 3.54167 -0.814286 5.5 1.5 4.66667 -1.7

##Experimentos con diferentes valores de N y M, para el cálculo de C y comparar los resultados con los obtenidos con el código secuencial CPU.

In [12]:
%%cuda
#include <iostream>
#include <chrono>
#include <cmath>

__global__ void vectorOperationKernel(float* A, float* B, float* C, int N, int M) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        if (static_cast<int>(ceil(A[i])) % 2 == 0) {
            C[i] = 0;
            for (int j = i * M; j < (i + 1) * M; ++j) {
                C[i] += A[j] * i + B[j];
            }
        } else {
            C[i] = 0;
            for (int j = i * M; j < (i + 1) * M; ++j) {
                C[i] += A[j] * i - B[j];
            }
        }
    }
}

void sequentialVectorOperation(float* A, float* B, float* C, int N, int M) {
    for (int i = 0; i < N; ++i) {
        if (static_cast<int>(ceil(A[i])) % 2 == 0) {
            C[i] = 0;
            for (int j = i * M; j < (i + 1) * M; ++j) {
                C[i] += A[j] * i + B[j];
            }
        } else {
            C[i] = 0;
            for (int j = i * M; j < (i + 1) * M; ++j) {
                C[i] += A[j] * i - B[j];
            }
        }
    }
}

int main() {
    // Aquí debes crear y asignar memoria para los vectores A, B y C
    // Llamar a las funciones CUDA y de manejo de memoria adecuadas
    return 0;
}

// Kernel para la operación vectorial utilizando memoria compartida
__global__ void vectorOperationWithSharedMemory(float* A, float* B, float* C, int N, int M) {
    __shared__ float shared_A[256];
    __shared__ float shared_B[256];

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < N) {
        shared_A[threadIdx.x] = A[idx];
        shared_B[threadIdx.x] = B[idx];
        __syncthreads();

        float result = 0;
        if (ceil(shared_A[threadIdx.x]) % 2 == 0) {
            for (int j = 0; j < M; ++j) {
                result += shared_A[j] * idx + shared_B[j];
            }
        } else {
            for (int j = 0; j < M; ++j) {
                result += shared_A[j] * idx - shared_B[j];
            }
        }

        C[idx] = result;
    }
}

// Kernel para la operación vectorial sin memoria compartida
__global__ void vectorOperationWithoutSharedMemory(float* A, float* B, float* C, int N, int M) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < N) {
        float result = 0;
        if (ceil(A[idx]) % 2 == 0) {
            for (int j = idx * M; j < (idx + 1) * M; ++j) {
                result += A[j] * idx + B[j];
            }
        } else {
            for (int j = idx * M; j < (idx + 1) * M; ++j) {
                result += A[j] * idx - B[j];
            }
        }

        C[idx] = result;
    }
}

int main() {
    // Tamaños de problema y de bloque
    int N_values[] = {20224, 40192, 80128, 100096};
    int M_values[] = {64, 128, 256};

    // Iterar sobre diferentes valores de N y M
    for (int i = 0; i < sizeof(N_values) / sizeof(N_values[0]); ++i) {
        for (int j = 0; j < sizeof(M_values) / sizeof(M_values[0]); ++j) {
            int N = N_values[i];
            int M = M_values[j];


/tmp/tmptvicp8p9/c044c5aa-37d1-40c1-9788-633aa56af32a/single_file.cu(57): error: expression must have integral or unscoped enum type
          if (ceil(shared_A[threadIdx.x]) % 2 == 0) {
              ^

/tmp/tmptvicp8p9/c044c5aa-37d1-40c1-9788-633aa56af32a/single_file.cu(77): error: expression must have integral or unscoped enum type
          if (ceil(A[idx]) % 2 == 0) {
              ^

/tmp/tmptvicp8p9/c044c5aa-37d1-40c1-9788-633aa56af32a/single_file.cu(91): error: function "main" has already been defined
  int main() {
      ^

At end of source: error: expected a "}"
/tmp/tmptvicp8p9/c044c5aa-37d1-40c1-9788-633aa56af32a/single_file.cu(98): note #3196-D: to match this "{"
          for (int j = 0; j < sizeof(M_values) / sizeof(M_values[0]); ++j) {
                                                                           ^

At end of source: error: expected a "}"
/tmp/tmptvicp8p9/c044c5aa-37d1-40c1-9788-633aa56af32a/single_file.cu(97): note #3196-D: to match this "{"
      for (int

In [ ]:
// Kernel para la operación vectorial sin memoria compartida
__global__ void vectorOperationWithoutSharedMemory(float* A, float* B, float* C, int N, int M) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < N) {
        float result = 0;
        if (ceil(A[idx]) % 2 == 0) {
            for (int j = idx * M; j < (idx + 1) * M; ++j) {
                result += A[j] * idx + B[j];
            }
        } else {
            for (int j = idx * M; j < (idx + 1) * M; ++j) {
                result += A[j] * idx - B[j];
            }
        }

        C[idx] = result;
    }
}

int main() {
    // Tamaños de problema y de bloque
    int N_values[] = {20224, 40192, 80128, 100096};
    int M_values[] = {64, 128, 256};

    // Iterar sobre diferentes valores de N y M
    for (int i = 0; i < sizeof(N_values) / sizeof(N_values[0]); ++i) {
        for (int j = 0; j < sizeof(M_values) / sizeof(M_values[0]); ++j) {
            int N = N_values[i];
            int M = M_values[j];


In [ ]:
 // Declarar y asignar memoria para los vectores en el host
            float *h_A, *h_B, *h_C_seq, *h_C_shared, *h_C_no_shared;
            h_A = new float[N];
            h_B = new float[N];
            h_C_seq = new float[N];
            h_C_shared = new float[N];
            h_C_no_shared = new float[N];

            // Inicializar los vectores en el host
            for (int i = 0; i < N; ++i) {
                h_A[i] = 1.5 * ((1 + 5 * i % 7) / (1 + i % 5));
                h_B[i] = 2 * ((2 + i % 5) / (1 + i % 7));
            }

            // Medir tiempo de ejecución secuencial
            auto start_time_seq = std::chrono::high_resolution_clock::now();
            sequentialVectorOperation(h_A, h_B, h_C_seq, N, M);
            auto end_time_seq = std::chrono::high_resolution_clock::now();
            std::chrono::duration<double> seq_execution_time = end_time_seq - start_time_seq;

            // Declarar y asignar memoria para los vectores en el dispositivo
            float *d_A, *d_B, *d_C_shared, *d_C_no_shared;
            cudaMalloc(&d_A, N * sizeof(float));
            cudaMalloc(&d_B, N * sizeof(float));
            cudaMalloc(&d_C_shared, N * sizeof(float));
            cudaMalloc(&d_C_no_shared, N * sizeof(float));

            // Copiar datos del host al dispositivo
            cudaMemcpy(d_A, h_A, N * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_B, h_B, N * sizeof(float), cudaMemcpyHostToDevice);

            // Configurar tamaños de bloque y de grid
            int blockSize = 256;
            int gridSize = (N + blockSize - 1) / blockSize;

            // Medir tiempo de ejecución con memoria compartida
            auto start_time_shared = std::chrono::high_resolution_clock::now();
            vectorOperationWithSharedMemory<<<gridSize, blockSize>>>(d_A, d_B, d_C_shared, N, M);
            cudaDeviceSynchronize();
            auto end_time_shared = std::chrono::high_resolution_clock::now();
            std::chrono::duration<double> shared_execution_time = end_time_shared - start_time_shared;

            // Medir tiempo de ejecución sin memoria compartida
            auto start_time_no_shared = std::chrono::high_resolution_clock::now();
            vectorOperationWithoutSharedMemory<<<gridSize, blockSize>>>(d_A, d_B, d_C_no_shared, N, M);
            cudaDeviceSynchronize();
            auto end_time_no_shared = std::chrono::high_resolution_clock::now();
            std::chrono::duration<double> no_shared_execution_time = end_time_no_shared - start_time_no_shared;

            // Copiar datos del dispositivo al host
            cudaMemcpy(h_C_shared, d_C_shared, N * sizeof(float), cudaMemcpyDeviceToHost);
            cudaMemcpy(h_C_no_shared, d_C_no_shared, N * sizeof(float), cudaMemcpyDeviceToHost);

            // Comparar resultados
            bool seq_vs_shared = true;
            bool seq_vs_no_shared = true;
            for (int i = 0; i < N; ++i) {
                if (std::abs(h_C_seq[i] - h_C_shared[i]) > 1e-5) {
                    seq_vs_shared = false;
                }
                if (std::abs(h_C_seq[i] - h_C_no_shared[i]) > 1e-5) {
                    seq_vs_no_shared = false;
                }
            }

            // Imprimir resultados
            std::cout << "N = " << N << ", M = " << M << std::endl;
            std::cout << "Tiempo de ejecución secuencial: " << seq_execution_time.count() << " segundos" << std::endl;
            std::cout << "Tiempo de ejecución con memoria compartida: " << shared_execution_time.count() << " segundos" << std::endl;
            std::cout << "Tiempo de ejecución sin memoria compartida: " << no_shared_execution_time.count() << " segundos" << std::endl;
            std::cout << "Comparación con cálculo secuencial (con memoria compartida): " << (seq_vs_shared ? "Correcta" : "Incorrecta") << std::endl;
            std::cout << "Comparación con cálculo secuencial (sin memoria compartida): " << (seq_vs_no_shared ? "Correcta" : "Incorrecta") << std::endl;

            // Liberar memoria
            delete[] h_A;
            delete[] h_B;
            delete[] h_C_seq;
            delete[] h_C_shared;
            delete[] h_C_no_shared;
            cudaFree(d_A);
            cudaFree(d_B);
            cudaFree(d_C_shared);
            cudaFree(d_C_no_shared);
        }
    }

    return 0;
}